# 株価推移

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hiohiohio/jquants-api-client-python/blob/feature/add-sample-notebooks/examples/20220825-001-price-movement.ipynb)

このノートブックでは指定した日付を基準として複数の銘柄の株価の推移をプロットします。


----

**このノートブックはGoogle Driveを使用します。**

- Google Drive の以下のファイルにリフレッシュトークンが書き込まれていることを想定しています。
    - `MyDrive/drive_ws/secret/jquantsapi-key.txt`
- Google Drive の以下のフォルダーにデータを書き込みます。
    - `MyDrive/drive_ws/marketdata`

In [ ]:
# 表示する銘柄のコードを指定します (e.g. 8697)
TARGET_SYMBOLS = [
    "7201",  # 日産
    "7203",  # トヨタ
    "7267",  # ホンダ
]

# 基準となる日付を指定します。(YYYY-MM-DD)
BASE_DATE = "2022-07-01"

In [ ]:
# 必要なモジュールをインストールします。
! python -m pip install jquants-api-client japanize-matplotlib

In [ ]:
# Google drive をマウントします。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from datetime import datetime

import japanize_matplotlib
import jquantsapi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# pandas の表示制限を調整します
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 2000)

In [ ]:
# TARGET_SYMBOLS が4桁で入力されている場合は末尾に0を付与する
for i in range(len(TARGET_SYMBOLS)):
    if len(TARGET_SYMBOLS[i]) == 4:
        TARGET_SYMBOLS[i] += "0"

In [ ]:
# 一度取得したデータは Google Drive 上に保存して再利用します。
# 保存先ディレクトリを指定します。
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"
os.makedirs(STORAGE_DIR_PATH, exist_ok=True)
STORAGE_DIR_PATH

In [ ]:
# J-Quants APIのトークンを保存してあるファイルを指定します
REFRESH_TOKEN_FILE_PATH = "/content/drive/MyDrive/drive_ws/secret/jquantsapi-key.txt"

In [ ]:
def get_refresh_token(refresh_token_file_path: str = REFRESH_TOKEN_FILE_PATH):
    with open(refresh_token_file_path, "r") as f:
        refresh_token = f.read()
    return refresh_token.rstrip().lstrip()

In [ ]:
# ファイルからリフレッシュトークンを読み込みます
refresh_token = get_refresh_token()

In [ ]:
# J-Quants APIクライアントを初期化します
jqapi = jquantsapi.Client(refresh_token=refresh_token)

In [ ]:
# 銘柄情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
if now.hour < 22:
    # データ更新時間前の場合は日付を1日ずらします。
    now -= pd.Timedelta(1, unit="D")
list_file = f"{STORAGE_DIR_PATH}/list_{now.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(list_file):
    df_list = jqapi.get_list()
    df_list.to_csv(list_file, compression="gzip", index=False)
    print(f"save file: {list_file}")

# ファイルからデータを読み込みます
print(f"file exists: {list_file}, loading")
df_list = pd.read_csv(list_file, dtype="str")

In [ ]:
# 取得したデータを表示して確認します
df_list.head(2)

In [ ]:
# 株価情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
start_dt = pd.Timestamp(BASE_DATE, tz="Asia/Tokyo")
end_dt = now
if end_dt.hour < 19:
    # データ更新時間前の場合は日付を1日ずらします。
    end_dt -= pd.Timedelta(1, unit="D")
price_file = f"{STORAGE_DIR_PATH}/price_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(price_file):
    df_p = jqapi.get_price_range(start_dt=start_dt, end_dt=end_dt)
    df_p.to_csv(price_file, compression="gzip", index=False)
    df_p.reset_index(drop=True, inplace=True)
    print(f"save file: {price_file}")

# データを読み込みます
print(f"file exists: {price_file}, loading")
df_p = pd.read_csv(price_file, dtype="str")
df_p.reset_index(drop=True, inplace=True)
# 各列のデータ型を調整します
df_p.loc[:, "Date"] = pd.to_datetime(df_p["Date"], format="%Y-%m-%d")
df_p.loc[:, "Open"] = df_p["Open"].astype(np.float64)
df_p.loc[:, "High"] = df_p["High"].astype(np.float64)
df_p.loc[:, "Low"] = df_p["Low"].astype(np.float64)
df_p.loc[:, "Close"] = df_p["Close"].astype(np.float64)
df_p.loc[:, "Volume"] = df_p["Volume"].astype(np.float64)
df_p.loc[:, "TurnoverValue"] = df_p["TurnoverValue"].astype(np.float64)
df_p.loc[:, "AdjustmentFactor"] = df_p["AdjustmentFactor"].astype(np.float64)
df_p.loc[:, "AdjustmentOpen"] = df_p["AdjustmentOpen"].astype(np.float64)
df_p.loc[:, "AdjustmentHigh"] = df_p["AdjustmentHigh"].astype(np.float64)
df_p.loc[:, "AdjustmentLow"] = df_p["AdjustmentLow"].astype(np.float64)
df_p.loc[:, "AdjustmentClose"] = df_p["AdjustmentClose"].astype(np.float64)
df_p.loc[:, "AdjustmentVolume"] = df_p["AdjustmentVolume"].astype(np.float64)

In [ ]:
# 取得したデータを表示して確認します
df_p.head(2)

In [ ]:
# 銘柄
target_symbols = TARGET_SYMBOLS
# 基準日
base_date = BASE_DATE

# 対象銘柄および基準日以降のデータに絞り込み (計算量を減少させるため)
df_work = df_p.loc[(df_p["Code"].isin(target_symbols)) & (df_p["Date"] >= base_date)].copy()

# # 株価をフィルするために並べ替えます
df_work.sort_values(["Code", "Date"], inplace=True)
# # 株価をフィルするために0をnanに置き換えます
df_work["AdjustmentClose"].replace({0.0: np.nan}, inplace=True)
# # 終値が0の場合は前営業日の終値を使用します
df_work.loc[:, "AdjustmentClose"] = df_work.groupby("Code")["AdjustmentClose"].ffill()
# # 終値がnanの場合は翌営業日の終値を使用します (データの先頭を処理します)
# df_work.loc[:, "AdjustmentClose"] = df_work.groupby("Code")["AdjustmentClose"].bfill()

# 基準日の終値を基準価格としてコピーします
df_work.loc[:, "base_price"] = df_work.loc[df_work["Date"] == base_date, "AdjustmentClose"]
# base_priceをフォーワードフィルします
df_work.loc[:, "base_price"] = df_work.loc[:, "base_price"].ffill()

# 基準価格からの変化率を計算
df_work.loc[:, "percentage"] = df_work["AdjustmentClose"].div(df_work["base_price"]) * 100

# 銘柄情報と結合します
df_work = pd.merge(df_work, df_list, on="Code")
# 銘柄コードと銘柄名を結合します
df_work["銘柄"] = df_work["CompanyName"] + "(" + df_work["Code"] + ")"

# プロット用に整形します
df = df_work.pivot(index='Date', columns='銘柄', values='percentage')

# プロットします
ax = df.plot(figsize=(20, 8), grid=True)

# 縦軸100の位置に基準線を描画します
ax.axhline(y=100, color="black")